### Feature: Como a planilha orçamentária foi modificada ao longo do tempo

### Features Extracted:

Issue: [#65](https://github.com/lappis-unb/salic-ml/issues/65)
- https://github.com/lappis-unb/salic-ml/issues/65

#### Recarregar automaticamente os módulos

In [1]:
%load_ext autoreload
%autoreload 2

### Importing data

In [2]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats


from salicml.utils.dates import Dates
from core.utils.read_csv import read_csv_with_different_type
from salicml.utils.utils import debug

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir, os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

#### PLANILHA ORCAMENTARIA

QUERY: /data/scripts/planilha_orcamentaria.sql


In [3]:
dt_aprovacao_pai = 'planilha_aprovacao_pai_head.csv'
usecols = ['PRONAC', 'idPlanilhaAprovacao', 'idPlanilhaAprovacaoPai',
       'Item', 'dtPlanilha', 'stAtivo', 'Segmento',]

dtype = {
    'PRONAC': str,
}
dt_aprovacao = read_csv_with_different_type(dt_aprovacao_pai, dtype, usecols=usecols)
dt_aprovacao['idPlanilhaAprovacaoPai'] = dt_aprovacao['idPlanilhaAprovacaoPai'].fillna(value=-1)
dt_aprovacao['idPlanilhaAprovacaoPai'] = dt_aprovacao['idPlanilhaAprovacaoPai'].round(0).astype(int)

display(dt_aprovacao.columns)
display(dt_aprovacao.dtypes)
dt_aprovacao.head()

Index(['PRONAC', 'idPlanilhaAprovacao', 'idPlanilhaAprovacaoPai', 'Item',
       'dtPlanilha', 'stAtivo', 'Segmento'],
      dtype='object')

PRONAC                    object
idPlanilhaAprovacao        int64
idPlanilhaAprovacaoPai     int64
Item                      object
dtPlanilha                object
stAtivo                   object
Segmento                  object
dtype: object

,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
0,095259,109198,-1,Contador,2011-03-14 15:31:56.000,S,77
1,093803,792707,-1,Concreto armado para reforços,2013-02-06 11:01:45.687,S,5E
2,093803,1692078,792707,Concreto armado para reforços,2015-11-18 15:11:35.507,N,5E
3,093803,1692134,792763,Madeira: tesoura,2015-11-18 15:11:35.687,N,5E
4,093803,792763,-1,Madeira: tesoura,2013-02-06 11:01:45.687,S,5E


### O caso de idPlanilhaAprovacaoPai = null

No banco de dados há diversos casos `idPlanilhaAprovacaoPai = null`. Há duas observações importantes sobre este caso:

- `idPlanilhaAprovacaoPai = null` identifica itens sem "pai", ou seja, são os itens que ainda não foram modificados, estão na sua primeira versão aprovada.

- O numpy interpreta valores numéricos `null` como `NaN`, o que são convertidos automaticamente para `float`. No banco de dados esta coluna é armazenada como `int`, e tratá-la como `float` é perigoso. Entretanto aparentemente o `numpy` não aceita valores `NaN` como `Int64`. Enfim a solução atual foi converter todos os valoes `NaN` para `-1`, e depois arredondar todos os valores float para `Int64`.

# Dividindo pronacs de treino e pronacs de teste

50/50

In [4]:
from sklearn.model_selection import train_test_split


all_pronacs = dt_aprovacao.PRONAC.unique()
pronacs_train, pronacs_test = train_test_split(all_pronacs, test_size=0.5)


dt_train = dt_aprovacao[dt_aprovacao['PRONAC'].isin(pronacs_train)].copy()
dt_test = dt_aprovacao[dt_aprovacao['PRONAC'].isin(pronacs_test)].copy()

In [5]:
print('pronacs_train = {}'.format(pronacs_train))
print('pronacs_test = {}'.format(pronacs_test))

assert (dt_train.shape[0] + dt_test.shape[0]) == dt_aprovacao.shape[0]

display(dt_train.head())
display(dt_test.head())

pronacs_train = ['1011833' '103228' '093803' '101120' '092907' '091695' '095195' '091102'
 '094464' '1012471' '097348']
pronacs_test = ['095259' '093392' '098671' '100855' '1012800' '093242' '110895' '104458'
 '110277' '110300' '101800']


,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
1,093803,792707,-1,Concreto armado para reforços,2013-02-06 11:01:45.687,S,5E
2,093803,1692078,792707,Concreto armado para reforços,2015-11-18 15:11:35.507,N,5E
3,093803,1692134,792763,Madeira: tesoura,2015-11-18 15:11:35.687,N,5E
4,093803,792763,-1,Madeira: tesoura,2013-02-06 11:01:45.687,S,5E
6,094464,53321,-1,Material de consumo,2011-02-07 17:24:32.000,S,55


,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
0,095259,109198,-1,Contador,2011-03-14 15:31:56.000,S,77
5,110895,124708,-1,Maestro,2011-04-05 15:11:20.000,S,85
9,110895,124726,-1,Cópia trailer,2011-04-05 15:11:20.000,S,85
10,110895,124759,-1,Coordenação Administrativo- Financeiro,2011-04-05 15:11:20.000,S,85
11,093242,37564,-1,Material de escritório,2011-02-07 17:04:15.000,S,51


In [6]:
dt_train.sort_values(by=['PRONAC'], inplace=True)
dt_train.head(100)

,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
56,091102,72679,-1,Auxiliar Administrativo,2011-02-23 19:29:24.000,S,71
94,091102,72708,-1,Locação de Gerador de Energia,2011-02-23 19:29:25.000,S,71
89,091102,72684,-1,Cópias e reproduções,2011-02-23 19:29:24.000,S,71
24,091102,72678,-1,Assistente administrativo,2011-02-23 19:29:24.000,S,71
68,091102,72710,-1,Locação de palco c/ cobertura,2011-02-23 19:29:25.000,S,71
57,091102,72688,-1,Material de escritório,2011-02-23 19:29:24.000,S,71
37,091102,72713,-1,Anúncio de 1/2 de página\r\n,2011-02-23 19:29:25.000,S,71
95,091102,72728,-1,Captação,2011-02-23 19:29:25.000,S,71
88,091695,116503,-1,Reparos e manutenção,2011-03-21 11:17:31.000,S,71
87,091695,116484,-1,Produtor,2011-03-21 11:17:31.000,S,71


In [7]:
dt_pronac = dt_train.groupby(by=['PRONAC'])

In [8]:
PRONAC = 'PRONAC'
ITEM_ID = 'idPlanilhaAprovacao'
PARENT_ID = 'idPlanilhaAprovacaoPai'
ST_ATIVO = 'stAtivo'
NULL_PARENT = -1

In [23]:
def print_tree(parent, node):
    print(node, end='')
    father = parent[node]
    if father != NULL_PARENT:
        print(' -> ', end='')
        print_tree(parent, father)
    else:
        print()  
        

def print_forest(pronac, parent, leaves):
    print('PRONAC = [{}]\n'.format(pronac))
    print('parent = {}\n'.format(parent))
    print('leaves = {}\n'.format(leaves))
    print('\n')
    for leaf in leaves:
        print_tree(parent, leaf)

In [24]:
def init_parent_data(rows):
    parent = {}
    leaves = rows.idPlanilhaAprovacao.unique()

    for index, row in rows.iterrows():
        pronac = row[PRONAC]
        item_id = row[ITEM_ID]
        parent_id = row[PARENT_ID]
        parent[item_id] = parent_id
        
        try:
            leaves.remove(parent_id)
        except:
            # parent_id was already removed before
            pass   
        
    return parent, leaves

pronac = '093803'
rows = dt_train[dt_train.PRONAC == pronac]
#display(rows)

parent, leaves = init_parent_data(rows)
print_forest(pronac, parent, leaves)

PRONAC = [093803]

parent = {792718: -1, 1692089: 792718, 792737: -1, 1692108: 792737, 792707: -1, 792811: -1, 1692182: 792811, 792736: -1, 1692107: 792736, 792767: -1, 1692138: 792767, 1692109: 792738, 1692073: 792702, 792738: -1, 792778: -1, 792702: -1, 1692149: 792778, 1692134: 792763, 792781: -1, 1692152: 792781, 1692146: 792775, 792792: -1, 792775: -1, 792809: -1, 1692180: 792809, 792763: -1, 1692078: 792707, 1692163: 792792}

leaves = [ 792718 1692089  792737 1692108  792707  792811 1692182  792736 1692107
  792767 1692138 1692109 1692073  792738  792778  792702 1692149 1692134
  792781 1692152 1692146  792792  792775  792809 1692180  792763 1692078
 1692163]



792718
1692089 -> 792718
792737
1692108 -> 792737
792707
792811
1692182 -> 792811
792736
1692107 -> 792736
792767
1692138 -> 792767
1692109 -> 792738
1692073 -> 792702
792738
792778
792702
1692149 -> 792778
1692134 -> 792763
792781
1692152 -> 792781
1692146 -> 792775
792792
792775
792809
1692180 -> 792809
792763
1692078 -

In [16]:
dt_train[dt_train.stAtivo == 'N'].head()

,PRONAC,idPlanilhaAprovacao,idPlanilhaAprovacaoPai,Item,dtPlanilha,stAtivo,Segmento
42,093803,1692089,792718,Escritórios,2015-11-18 15:11:35.543,N,5E
44,093803,1692108,792737,Instalação provisória de força de luz,2015-11-18 15:11:35.603,N,5E
52,093803,1692182,792811,Correios,2015-11-18 15:11:35.850,N,5E
80,093803,1692107,792736,Instalação provisória de água e esgoto,2015-11-18 15:11:35.600,N,5E
82,093803,1692138,792767,Mestre de obra,2015-11-18 15:11:35.700,N,5E


In [12]:
segment_projects = dt_train[['PRONAC', 'idSegmento', 'idComprovantePagamento']].groupby(['idSegmento', 'PRONAC']).nunique()
segment_projects.drop(columns=['PRONAC', 'idSegmento'], inplace=True)
segment_projects.rename(columns={'idComprovantePagamento': 'NumeroComprovantes'}, inplace=True)

display(segment_projects.head())

KeyError: "['idSegmento' 'idComprovantePagamento'] not in index"

### Plotando as distribuiçoes dos segmentos mais comuns

In [ ]:
common_segements = segment_projects.groupby(['idSegmento']).count()
common_segements.rename(columns={'NumeroComprovantes': 'NumeroProjetos'}, inplace=True)
common_segements.sort_values(by=['NumeroProjetos'], ascending=False, inplace=True)
display(common_segements.head())

In [ ]:
from scipy.stats import norm


def plot_segment(id, receipts):
    plt.hist(receipts, bins=100, density=True, stacked=True, alpha=0.6, color='g', edgecolor='black')
    #plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.title('Segmento "{}"'.format(id))
    plt.xlabel('Número de recibos por projeto')
    plt.ylabel('Frequência de projetos')

    mu, std = np.mean(receipts), np.std(receipts)
    
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 200)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2)
    plt.show()

for counter, id_segmento in enumerate(common_segements.index.values):    
    projects_total_receipts = segment_projects.loc[id_segmento].NumeroComprovantes.values
    plot_segment(id_segmento, projects_total_receipts)
    
    counter += 1
    if counter == 20:
        break

In [ ]:
arr = segment_projects['NumeroComprovantes'].values
plot_segment('all', arr)

In [ ]:
segment_receipts_avg_std = segment_projects.groupby(['idSegmento'])
segment_receipts_avg_std = segment_receipts_avg_std.agg(['count', 'sum', 'mean', 'std'])

segment_receipts_avg_std.columns = segment_receipts_avg_std.columns.droplevel(0)

display(segment_receipts_avg_std.head())

# Calculando a porcentagem de outliers no conjunto de teste

In [ ]:
project_receipts_grp = dt_items[['PRONAC', 'idComprovantePagamento', 'idSegmento']].groupby(['PRONAC'])
project_receipts = project_receipts_grp.nunique()
project_receipts.drop(columns=['PRONAC', 'idSegmento'], inplace=True)
project_receipts.rename(columns={'idComprovantePagamento': 'NumeroComprovantes'}, inplace=True)
display(project_receipts.head())

In [ ]:
id_segmento = '11'
display(segment_receipts_avg_std.loc[id_segmento])
mean = segment_receipts_avg_std.loc[id_segmento]['mean']
print('mean = {}'.format(mean))

In [ ]:
from salicml.outliers.gaussian_outlier import is_outlier


def is_total_receipts_outlier(pronac):
    assert isinstance(pronac, int)
    
    total_receipts = project_receipts.loc[pronac]['NumeroComprovantes']
    id_segmento = project_receipts_grp.get_group(pronac).iloc[0]['idSegmento']
    
    if not np.isin(id_segmento, segment_receipts_avg_std.index):
        raise ValueError('Segment {} was not trained'.format(id_segmento))
    
    mean = segment_receipts_avg_std.loc[id_segmento]['mean']
    std = segment_receipts_avg_std.loc[id_segmento]['std']
    outlier = is_outlier(total_receipts, mean, std)
    return outlier

pronac = int(np.random.choice(dt_test.PRONAC.values))
print('pronac = {}'.format(pronac))
is_total_receipts_outlier(pronac)

In [ ]:
from salicml.outliers.gaussian_outlier import outlier_probability


print(outlier_probability(5011203123, 113132, c=1.5))
print(outlier_probability(5, 132, c=1.5))
print(outlier_probability(-1, 3, c=1.5))

In [ ]:
pronacs_test = dt_test.PRONAC.unique()

outlier_arr = []

for i, pronac in enumerate(pronacs_test):
    try:
        outlier = is_total_receipts_outlier(int(pronac))
        outlier_arr.append(1.0 if outlier else 0.0)
    except ValueError as err:
        print(err)
    
describe = pd.DataFrame(outlier_arr).describe()
display(describe)

# Análise dos resultados

Para `c = 1.5`, esperava-se que `6.68%` dos projetos fossem considerados outliers em termos dos números de itens por projeto. Entretanto, para os conjuntos de treino e teste utilizados e `c = 1.5`, `7.57%`dos projetos foram considerados outliers.

# Contribuições futuras

A celula anterior demonstra um problema na divisão dos pronacs nos conjuntos de treino e teste: alguns segmentos, por exemplo o segmento `9I` ficaram fora do conjunto do treino, o que não permite detectar anomalias para este segmento e os demais que não foram incluidos no conjunto de treino. Portanto uma contribução futura desejável é que o conjunto de treino garanta que para segmento, pelo menos 50% do total dos projetos para aquele segmento esteja dentro do conjunto de pronacs de treino.